In [1]:
import datetime as dt
import datadotworld as dw
import numpy as np
import pandas as pd

import plotly as py
import plotly.graph_objs as go
import plotly.offline as pyo

In [2]:
pyo.init_notebook_mode(connected=True)

In [3]:
def fiscal_year(df):
    """
    Function takes a dataframe with a DateTimeIndex and
    produces list with the corresponding fiscal year as a
    four digit year for each date on the index of the dataframe.

    The function is based on the Maryland Govt fiscal year which
    runs from July 1st to June 30th.  It returns a list that is the
    same size as the original dataframe and allows the function call
    to be passed as a new column for fiscal year.
    """
    fiscal_year = np.where(df.index.month >= 7,df.index.year+1, df.index.year)
    return fiscal_year

In [4]:
# load dataset from data.world service
dataset = dw.load_dataset('dgs-kpis/fmd-maintenance')
archibus_data = dataset.dataframes['archibus_maintenance_data']

/Users/babilalima/Documents/Work_Coding_Projects/data-validation/Data-Validation/venv/lib/python3.6/site-packages/datadotworld/models/dataset.py:194: UserWarning:

Unable to set data frame dtypes automatically using archibus_maintenance_data schema. Data types may need to be adjusted manually. Error: Integer column has NA values in column 50

/Users/babilalima/Documents/Work_Coding_Projects/data-validation/Data-Validation/venv/lib/python3.6/site-packages/datadotworld/util.py:118: DtypeWarning:

Columns (6,8,30,32,34,38,49,53,56,60,68,74,83,85,86,88,90,91) have mixed types. Specify dtype option on import or set low_memory=False.



In [5]:
# target_columns = ['wo_id','date_completed','prob_type',
#                    'bl_id','completed_by','time_completed','time_start','time_end']

In [6]:
def get_dataframe(data_source):
    '''
    
    '''
    target_columns = (['wo_id','date_completed','prob_type','bl_id','completed_by',
                        'date_requested','time_completed','time_start','time_end'])
    if isinstance(data_source, str):
        df = pd.read_excel(io=data_source)
        df = df[target_columns][(df['prob_type'] != 'TEST(DO NOT USE)')]
        df['date_requested'] = pd.to_datetime(df['date_requested'])
        df.set_index('date_requested', inplace=True)
        df['duration'] = df['date_completed'] - df.index
        df['fiscal_year_requested'] = fiscal_year(df)  
        df['fiscal_year_completed'] =  [date.year + 1 if date.month >= 7 else date.year for date in df['date_completed']]
        df.sort_index(inplace=True)
        
    elif isinstance(data_source, dw.models.dataset.LocalDataset):
        df = data_source.dataframes['archibus_maintenance_data']
        df = df[target_columns][(df['prob_type'] != 'TEST(DO NOT USE)')]        
        df['date_completed'] = pd.to_datetime(df['date_completed'])
        df['date_requested'] = pd.to_datetime(df['date_requested'])
        df.set_index('date_requested', inplace=True)
        df['duration'] = df['date_completed'] - df.index
        df['fiscal_year_requested'] = np.where(df.index.month >= 7,df.index.year+1, df.index.year)
        df['fiscal_year_completed'] =  [date.year + 1 if date.month >= 7 else date.year for date in df['date_completed']]
        df.sort_index(inplace=True)
        
    else:
        print("""
Function expects type io string or datadotworld.models.dataset.LocalDataset 
but got {}""".format(type(data_source)))
        
    return df
    

In [7]:
df = get_dataframe(dataset)

In [8]:
fy_list, fy_dfs = [],[]
for year in df['fiscal_year_requested'].unique():
    fy_dfs.append(df.loc[(df['fiscal_year_requested']==year)])
    fy_list.append(year)
fy_dfs = dict(zip(fy_list,fy_dfs))

In [9]:
# get total number of work orders requested in each fiscal year
work_order_volume = []
for key, value in fy_dfs.items():
    work_order_volume.append(value.wo_id.count())
work_order_volume = dict(zip(fy_list,work_order_volume))
work_order_volume

{2014: 12209, 2015: 14522, 2016: 16679, 2017: 17873, 2018: 16593, 2019: 6186}

In [10]:
closed_workorders_dfs = []
for key, value in fy_dfs.items():
    closed_workorders_dfs.append(value[value['date_completed'].notnull()])    
closed_workorders_dfs = dict(zip(fy_dfs.keys(), closed_workorders_dfs))

## for budget narrative

In [31]:
df['prob_type'].nunique()

103

In [33]:
df.head()

,wo_id,date_completed,prob_type,bl_id,completed_by,time_completed,time_start,time_end,duration,fiscal_year_requested,fiscal_year_completed
date_requested,,,,,,,,,,,
2013-10-16,19.0,2014-03-12,HVAC,B04004,BONNIE.MILLER,11:48:49,NaN,NaN,147 days,2014,2014.0
2013-10-16,2.0,2013-10-23,FENCE_GATE,B00163,ANN.BRAUN,07:54:25,NaN,NaN,7 days,2014,2014.0
2013-10-16,16.0,2014-01-08,CEILTILE,B00056,JIMMY.HOLTHAUS,14:08:09,NaN,NaN,84 days,2014,2014.0
2013-10-16,11.0,2013-10-16,PLUMB/OTHER,B00020,CHARLES.CLARIDY,13:06:44,NaN,NaN,0 days,2014,2014.0
2013-10-16,20.0,2013-10-23,CEILTILE,B08535,ANTHONY.DINISIO,12:30:12,NaN,NaN,7 days,2014,2014.0


In [36]:
df[df['fiscal_year_requested']==2018]['prob_type'].nunique()

89

In [41]:
df[(df['fiscal_year_requested']==2017) & 
   (df['fiscal_year_completed']==2017)]['duration'].mean().days

18

In [42]:
df[(df['fiscal_year_requested']==2018) & 
   (df['fiscal_year_completed']==2018)]['duration'].mean().days

19

In [43]:
corrective_maintenance = ['BOILER','CHILLERS','COOLING TOWERS','HVAC',
                                'HVAC INFRASTRUCTURE','HVAC|REPAIR']
preventative_maintenance = ['PREVENTIVE MAINT','HVAC|PM']

# <font color=red>answers budget narrative for pm:cm</font>

In [47]:
df[(df['fiscal_year_requested']==2017) & 
   (df['fiscal_year_completed']==2017) & 
  (df['prob_type'].isin(corrective_maintenance))]['wo_id'].count()

1958

In [48]:
df[(df['fiscal_year_requested']==2018) & 
   (df['fiscal_year_completed']==2018) & 
  (df['prob_type'].isin(corrective_maintenance))]['wo_id'].count()

2443

In [49]:
2443 - 1958


485

In [61]:
environmental_types = ['ENVIR/ASBESTOS','ENVIR/LEADPAINT','ENVIR/MOLD',
                      'NVIR/RADON','ENVIRONMENTAL','NVIRONMENTAL|FIRE RESTORATION',
                      'ENVIRONMENTAL|FLOOD RESTORATION']

In [73]:
df[(df['fiscal_year_requested']==2017) & 
   (df['fiscal_year_completed']==2017) & 
  (df['prob_type'].isin(environmental_types))]['duration'].mean().days

56

In [70]:
df[(df['fiscal_year_requested']==2018) & 
   (df['fiscal_year_completed']==2018) & 
  (df['prob_type'].isin(environmental_types))]['duration'].mean().days

49

In [53]:
np.sort(df['prob_type'].unique())

array(['ADA', 'ADA REQUESTS', 'ADMINISTRATION', 'AIR QUALITY',
       'APPLIANCE', 'ASBESTOS', 'BATHROOM_FIXT', 'BOILER',
       'BUILDING EXTERIOR', 'BUILDING INTERIOR INSPECTION', 'BUILDING PM',
       'CARPENTRY', 'CB', 'CEILTILE', 'CHILLERS', 'COOLING TOWERS',
       'DELIVERY', 'DESIGN/RENOVATION', 'DOOR', 'DUCT CLEANING',
       'ELEC/EMERLGHT', 'ELEC/GENERAL', 'ELEC/LIGHT', 'ELEC/MOTOR',
       'ELEC/OTHER', 'ELEC/SECURITY', 'ELECTRICAL', 'ELEVATOR',
       'EMERGENCY SIGNAGE', 'ENVIR/ASBESTOS', 'ENVIR/LEADPAINT',
       'ENVIR/MOLD', 'ENVIR/RADON', 'ENVIRONMENTAL',
       'ENVIRONMENTAL|FIRE RESTORATION',
       'ENVIRONMENTAL|FLOOD RESTORATION', 'FENCE_GATE',
       'FIRE SUPPRESSION-PROTECTION', 'FLOOR', 'FUEL INSPECTION',
       'GENERATOR PM', 'HVAC', 'HVAC INFRASTRUCTURE', 'HVAC|PM',
       'HVAC|REPAIR', 'HVAC|REPLACEMENT', 'INSPECTION', 'LANDSCAPING',
       'LAWN', 'LOCK', 'OTHER', 'OUTLETS', 'OVERHDDOOR', 'PAINT',
       'PAINTING', 'PEDESTRIAN DOORS', 'PLUMB/LEAK', 'P

In [11]:
# add data for work order volume by problem type per fiscal year
# add average duration for each problem type per fiscal year
# add indicator for whether work order was on time or not

for key, value in closed_workorders_dfs.items():
    value['count'] = value.groupby(['prob_type'])['prob_type'].transform('count')
    value['avg_duration'] = (value.groupby(['prob_type'])['duration'].\
                        transform('sum') / value['count'])
    value['on_time'] = np.where(value.duration <= value.avg_duration,'hit','miss')


/Users/babilalima/Documents/Work_Coding_Projects/data-validation/Data-Validation/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/babilalima/Documents/Work_Coding_Projects/data-validation/Data-Validation/venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/babilalima/Documents/Work_Coding_Projects/data-validation/Data-Validation/venv/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is 

## PM to CM

In [12]:
# define filters for calculating PM : CM indicator
corrective_maintenance = ['BOILER','CHILLERS','COOLING TOWERS','HVAC',
                                'HVAC INFRASTRUCTURE','HVAC|REPAIR']
preventative_maintenance = ['PREVENTIVE MAINT','HVAC|PM']
hvac_problem_types = corrective_maintenance + preventative_maintenance

kpi_values_dict = {}
for key, value in fy_dfs.items():
        kpi_values_dict[key] = (
        value[value['prob_type'].isin(preventative_maintenance)]['prob_type'].
        value_counts().sum() / 
    
        value[value['prob_type'].isin(corrective_maintenance)]['prob_type'].
        value_counts().sum() * 100 )

In [13]:
trace1 = go.Bar(
    x = [year for year in kpi_values_dict.keys()],
    y = [value for value in kpi_values_dict.values()],
    marker = dict(color = '#3c5a89'),
    name = 'pm / cm'
                )

trace2 = go.Bar(
    x = [year for year in work_order_volume.keys()],
    y = [value for value in work_order_volume.values()],
    marker = {'color': 'grey',
             'line':{'color': 'black',
                    'width': 1.5}},
    opacity = .2,
    yaxis = 'y2',
    name = 'total work orders'
                )

layout = go.Layout(
        hovermode = 'closest',
        legend = dict(orientation = 'h'),
        title =' HVAC PM:CM KPI<br>FY{} -- FY{}'.format(min([year for year in kpi_values_dict.keys()]),max([year for year in kpi_values_dict.keys()])),
        yaxis=dict(title='pct %',
                   showgrid = False,
                   titlefont = dict(color = '#3c5a89'),
                   tickfont = dict(color = '#3c5a89')),
        yaxis2=dict(title='total work order volume',
                    showgrid = False,
                    titlefont=dict(color='grey'),
            tickfont=dict(
                color='grey'
            ),
            overlaying='y',
            side='right'
        ),
        plot_bgcolor = '#efefea',
        paper_bgcolor = '#efefea'
    )

fig = dict(data=[trace1,trace2], layout=layout)
pyo.iplot(fig)

In [14]:
# plot monthly
trace1 = go.Scatter(
    x = [quarter for quarter in df.resample('M')['wo_id'].count().index],
    y = [value for value in df.resample('M')['wo_id'].count()],
    mode = 'lines',
    name = 'monthly work orders',
    line = {'color': '#3c5a89',
           'width': 3,
           'dash': 'dot'}
                    )
trace2 = go.Scatter(
    x = [year for year in work_order_volume.keys()],
    y = [value for value in work_order_volume.values()],
    mode = 'lines',
    name = 'annual work orders',
    line = {'color': 'red',
           'width': 2},
    yaxis = 'y2')
    
layout = go.Layout(
        hovermode = 'closest',
        legend = dict(orientation = 'h'),
        title ='Work Order Volume<br>FY{} -- FY{}'.format(min(df['fiscal_year_requested']),
                                                          max(df['fiscal_year_requested'])),
        yaxis = dict(title='requests (monthly)<br>calendar year',
                   showgrid = False,
                   titlefont = dict(color = '#3c5a89'),
                   tickfont = dict(color = '#3c5a89')),
    
        yaxis2=dict(title='requests (annually)<br>fiscal year',
                    showgrid = False,
                    titlefont=dict(color='red'),
                    tickfont=dict(color='red'),
                    overlaying='y',
                    side='right'
                    ),
        plot_bgcolor = '#efefea',
        paper_bgcolor = '#efefea'
    )
fig = dict(data=[trace1, trace2], layout=layout)
pyo.iplot(fig)

In [15]:
# # plot volume over time y2  and count missed, pct on time, count on time y1

# kpi_values_dict = {}
# for year in closed_workorders_dfs.keys():
#     kpi_values_dict[year] = closed_workorders_dfs[year]['on_time'].\
#                             value_counts()[0] / fy_dfs[year]['wo_id'].count() * 100

# y1_trace = go.Scatter(
#     x = [year for year in kpi_values_dict.keys()],
#     y = [value for value in kpi_values_dict.values()],
#     mode = 'lines',
#     name = 'on-time %',
#     line = {'color': 'blue'}
# )

# y2_trace = go.Scatter(
#     x = [year for year in work_order_volume.keys()],
#     y = [value for value in work_order_volume.values()],
#     mode = 'lines',
#     name = 'annual work orders',
#     line = {'color': 'darkgrey',
#            'width': 4},
#     yaxis = 'y2')


# layout = go.Layout(
#     hovermode = 'closest',
#     legend = {'orientation': 'h'},
#     title = 'On Time Overview',
#     yaxis = dict(title = 'work orders closed<br> on time (pct)',
#                 showgrid = False,
#                 titlefont = dict(color='blue')),
    
#     yaxis2=dict(title='number requests',
#                     showgrid = False,
#                     titlefont=dict(color='darkgrey'),
#                     tickfont=dict(color='darkgrey'),
#                     overlaying='y',
#                     side='right'),
#     plot_bgcolor = '#efefea',
#     paper_bgcolor = '#efefea'
#                     )
# fig ={'data':[y1_trace, y2_trace], 'layout': layout}
# pyo.iplot(fig)



In [16]:
# bar charts of pct closed on time vs volume
kpi_values_dict = {}
for year in closed_workorders_dfs.keys():
    kpi_values_dict[year] = closed_workorders_dfs[year]['on_time'].\
                            value_counts()[0] / fy_dfs[year]['wo_id'].count() * 100


y1_trace = go.Bar(
    x = [year for year in kpi_values_dict.keys()],
    y = [value for value in kpi_values_dict.values()],
    name = 'on-time %',
    marker = dict(color = '#3c5a89'),
    text = ['{:.0f}%'.format(value) for value in kpi_values_dict.values()], 
    opacity = .7
                    )

y2_trace = go.Bar(
    x = [year for year in work_order_volume.keys()],
    y = [value for value in work_order_volume.values()],
    name = 'annual work orders',
    marker = {'color': 'grey'},
    text = ['{:,}'.format(value) for value in work_order_volume.values()],
    opacity = .8,
    yaxis = 'y2')


layout = go.Layout(
    hovermode = 'closest',
    legend = {'orientation': 'h'},
    title = 'On Time Overview',
    xaxis = {'title': 'fiscal year'},
    yaxis = dict(title = 'work orders closed<br> on time (pct)',
                showgrid = False,
                titlefont = dict(color='blue')),
    
    yaxis2=dict(title='number requests',
                    showgrid = False,
                    titlefont=dict(color='darkgrey'),
                    tickfont=dict(color='darkgrey'),
                    overlaying='y',
                    side='right'),
    plot_bgcolor = '#efefea',
    paper_bgcolor = '#efefea'
                    )
fig ={'data':[y1_trace, y2_trace], 'layout': layout}
pyo.iplot(fig)



In [17]:
def weekday_name(integer):
    """function takes integer from dayofweek value for timestamp and returns
    the name of the day of the week. 
    example: a day of week value of 0 returns 'Monday'
    """
    
    day_names = ("Monday","Tuesday","Wednesday","Thursday",
                     "Friday","Saturday","Sunday")
    return day_names[integer]

# filter out still open work orders
filtered_df = df[df['date_completed'].notnull()]
filtered_df['completed_day_name'] = filtered_df['date_completed'].apply(
    lambda x: weekday_name(x.dayofweek))

z_dict = {}
x = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

for year in filtered_df['fiscal_year_completed'].unique():
    z_dict[year] = []
    for day in x:
        z_dict[year].append(
            filtered_df[(filtered_df['fiscal_year_completed'] == year) & 
                          (filtered_df['completed_day_name'] == day)]['wo_id'].count()
                            )   

trace = go.Heatmap(
    z = [value for value in z_dict.values()],
    y = ['FY {}'.format(int(year)) for year in z_dict.keys()],
    x = x,
    colorscale = 'Portland')

layout = {'title': 'Number of Work Orders Closed Daily'}
pyo.iplot({'data':[trace],'layout':layout})

/Users/babilalima/Documents/Work_Coding_Projects/data-validation/Data-Validation/venv/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [18]:
# pct of of total work orders closed daily aggregated 
# at day level (by fiscal year)

z_dict = {}
x = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

for year in filtered_df['fiscal_year_completed'].unique():
    z_dict[year] = []
    for day in x:
        z_dict[year].append(
            (filtered_df[(filtered_df['fiscal_year_completed'] == year) &
                         (filtered_df['completed_day_name'] == day)]['wo_id'].count() /
             filtered_df[filtered_df['fiscal_year_completed'] == year]['wo_id'].count() * 100)
                            ) 

trace = go.Heatmap(
    z = [value for value in z_dict.values()],
    y = ['FY {}'.format(int(year)) for year in z_dict.keys()],
    x = x,
    colorscale = 'Cividis')

layout = {'title': 'Percentage of Work Orders Closed<br>by Day per Fiscal Year'}
pyo.iplot({'data':[trace],'layout':layout})

In [19]:
# distribution of number of work orders closed (agg by month) per fiscal year
def month_name(integer):
    """function takes integer from month value for timestamp 
    and returns the name of the month as a string. 
    Example: a timestamp with datetime.month value of 0 returns 'January'
    """
    month_names = ['January','February','March','April','May','June',
                    'July','August','September','October','November','December']
    return month_names[integer-1]

# filter out still open work orders
filtered_df = df[df['date_completed'].notnull()]
filtered_df['completed_month_name'] = [month_name(date) for date in filtered_df['date_completed'].dt.month]

z_dict = {}
x = ['January','February','March','April','May','June',
     'July','August','September','October','November','December']

for year in filtered_df['fiscal_year_completed'].unique():
    z_dict[year] = []
    for month in x:
        z_dict[year].append(
            filtered_df[(filtered_df['fiscal_year_completed'] == year) & 
                          (filtered_df['completed_month_name'] == month)]['wo_id'].count()
                            )   

trace = go.Heatmap(
    z = [value for value in z_dict.values()],
    y = ['FY {}'.format(int(year)) for year in z_dict.keys()],
    x = x,
    colorscale = 'Portland')

layout = {'title': 'Distribution of Work Orders Closed Monthly'}
pyo.iplot({'data':[trace],'layout':layout})

/Users/babilalima/Documents/Work_Coding_Projects/data-validation/Data-Validation/venv/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [20]:
# percentage of work orders closed (agg by month) per fiscal year
def month_name(integer):
    """function takes integer from month value for timestamp 
    and returns the name of the month as a string. 
    Example: a timestamp with datetime.month value of 0 returns 'January'
    """
    month_names = ['January','February','March','April','May','June',
                    'July','August','September','October','November','December']
    return month_names[integer-1]

# filter out still open work orders
filtered_df = df[df['date_completed'].notnull()]
filtered_df['completed_month_name'] = [month_name(date) for date in filtered_df['date_completed'].dt.month]

z_dict = {}
x = ['January','February','March','April','May','June',
     'July','August','September','October','November','December']

for year in filtered_df['fiscal_year_completed'].unique():
    z_dict[year] = []
    for month in x:
        z_dict[year].append(
            (filtered_df[(filtered_df['fiscal_year_completed'] == year) &
                         (filtered_df['completed_month_name'] == month)]['wo_id'].count() /
             filtered_df[filtered_df['fiscal_year_completed'] == year]['wo_id'].count() * 100)
                            )  

trace = go.Heatmap(
    z = [value for value in z_dict.values()],
    y = ['FY {}'.format(int(year)) for year in z_dict.keys()],
    x = x,
    colorscale = 'Cividis')

layout = {'title': 'Percentage of Work Orders Closed<br>Montly by Fiscal Year'}
pyo.iplot({'data':[trace],'layout':layout})

/Users/babilalima/Documents/Work_Coding_Projects/data-validation/Data-Validation/venv/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [21]:
# show requested vs closed in month as pct of total requested 
trace1 = go.Bar(
    x = [date for date in df[df['date_completed'].notnull()].resample('M')['date_completed'].count().index],
    y = [value for value in (df[df['date_completed'].notnull()].resample('M')['date_completed'].count() / 
                             df.resample('M')['wo_id'].count() * 100)],
    name = 'completed',
    marker = dict(color = '#87CD56'),
                )
trace2 = go.Bar(
    x = [date for date in df.resample('M')['wo_id'].count().index],
    y = [100 - value for value in (df[df['date_completed'].notnull()].resample('M')['date_completed'].count() / 
                                   df.resample('M')['wo_id'].count() * 100)],
    name = 'requested',
    marker = {'color': '#C70039'},
    
)

layout = go.Layout(
    hovermode = 'closest',
    title = 'Pct Work Orders Closed <i>vs</i><br>Opened Each Month',
    yaxis = {'title': 'percent'},
    xaxis = {'title': 'fiscal year'},
    barmode = 'stack'
                    )

fig = {'data': [trace1,trace2], 'layout': layout}
pyo.iplot(fig)

In [22]:

# create dictionary of filtered dataframes for top problem types for each year
dataframe_filter_lists, filtered_dataframes = [],{}
yearly_avg_durations = {} 
for i, num in enumerate([3,5,10,15]):
    # create filter as list object and use loop counter to acess index of list filter
    # to create dataframe with the top X filter
    dataframe_filter_lists.append(df['prob_type'].value_counts().head(num).index)
    filtered_dataframes['top {}'.format(str(num))] = df[(df['prob_type'].isin(dataframe_filter_lists[i]))]
    
    for year in filtered_dataframes['top {}'.format(str(num))]['fiscal_year_requested'].unique():        
        yearly_avg_durations['top: {} year: {}'.format(num,year)] = (
            filtered_dataframes['top {}'.format(str(num))][(filtered_dataframes['top {}'.format(str(num))]['fiscal_year_requested'] == year) &
                                                          (filtered_dataframes['top {}'.format(str(num))]['fiscal_year_completed'] == year)]['duration'].mean())


# for comparison add key value for all problem types (not just top 3,5,10,15 as filtered above) 
for year in df['fiscal_year_requested'].unique():
    yearly_avg_durations['all {}'.format(year)] = (df[(df['fiscal_year_requested'] == year) & 
                                     (df['fiscal_year_completed'] == year)]['duration'].mean())

# trace for all work orders for each fiscal year
trace_all = go.Bar(
    x = [text.split()[-1] for text in yearly_avg_durations.keys()],
    y = [value.days for key,value in yearly_avg_durations.items() if 'all' in key],
    name = 'all<br>work orders',
    marker = {'color': '#3c5a89'},
    text = ['All:<br> {} days'.format(value.days) for key,value in yearly_avg_durations.items() if 'all' in key],
                )   
# traces for top 3, top 5, top 10, top 15 work orders per fiscal year
trace_topFilters = []
for num, color in zip([3,5,10,15],['#D3DEE4','#849DAB','#589ABF','#04A1F8']):
    trace_topFilters.append(go.Bar(
        x = [text.split()[-1] for text in yearly_avg_durations.keys()],
        y = [value.days for key,value in yearly_avg_durations.items() if 'top: {} year'.format(num) in key ],
        name = 'Top {}<br>work orders'.format(num),
        marker = {'color': color},
        text = ['Top {}:<br> {} days'.format(num,value.days) for key,value in yearly_avg_durations.items() if 'top: {} year'.format(num) in key],
                    )) 
# traces for annual work order reqeust volume   
trace_requestVolume = go.Scatter(
    x = [year for year in df.groupby('fiscal_year_requested')['wo_id'].count().index],
    y = [value for value in df.groupby('fiscal_year_requested')['wo_id'].count()],
    mode = 'lines',
    name = 'requests',
    line = {'color': '#C70039',
           'width': 3},
    yaxis = 'y2')

layout = go.Layout(
    legend = {'orientation': 'h'},
    hovermode = 'closest',
    title = 'Average Work Order Durations<br><i>requested & completed within same fiscal year</i>',
    yaxis = {'title': 'days',
            'showgrid': False,
            'titlefont': {'color': '#3c5a89'},
            'tickfont': {'color': '#3c5a89'}},
    
    yaxis2 = {'title': 'work order<br>volume',
             'showgrid': False,
             'titlefont': {'color': '#C70039'},
             'tickfont': {'color': '#C70039'},
             'overlaying': 'y',
             'side': 'right'},
    
    plot_bgcolor = '#efefea',
    paper_bgcolor = '#efefea'
    
                    )
traces = trace_topFilters
for trace in [trace_all, trace_requestVolume]:
    traces.append(trace)

fig = {'data':traces, 'layout':layout}
pyo.iplot(fig)

In [23]:
df[(df['fiscal_year_requested']==2019) & 
  (df['fiscal_year_completed']==2019)]

,wo_id,date_completed,prob_type,bl_id,completed_by,time_completed,time_start,time_end,duration,fiscal_year_requested,fiscal_year_completed
date_requested,,,,,,,,,,,
2018-07-01,106281.0,2018-07-01,OTHER,B00061,NaN,13:59:13,NaN,NaN,0 days,2019,2019.0
2018-07-01,106279.0,2018-07-01,OTHER,B00057,NaN,13:45:59,NaN,NaN,0 days,2019,2019.0
2018-07-01,106280.0,2018-07-01,OTHER,B00056,NaN,13:46:50,NaN,NaN,0 days,2019,2019.0
2018-07-01,106276.0,2018-07-11,PREVENTIVE MAINT,B00063,NaN,08:07:34,NaN,NaN,10 days,2019,2019.0
2018-07-01,106284.0,2018-07-05,BATHROOM_FIXT,B00163,NaN,14:03:14,NaN,NaN,4 days,2019,2019.0
2018-07-01,106283.0,2018-08-17,BATHROOM_FIXT,B00163,NaN,13:58:05,NaN,NaN,47 days,2019,2019.0
2018-07-01,106285.0,2018-08-16,OTHER,B00163,NaN,11:22:29,NaN,NaN,46 days,2019,2019.0
2018-07-01,106277.0,2018-08-01,PLUMB/OTHER,B00057,NaN,13:39:27,NaN,NaN,31 days,2019,2019.0
2018-07-01,106278.0,2018-07-17,PLUMB/OTHER,B00056,NaN,09:41:55,NaN,NaN,16 days,2019,2019.0


In [24]:
yearly_avg_durations

{'top: 3 year: 2014': Timedelta('16 days 08:37:38.665922'),
 'top: 3 year: 2015': Timedelta('14 days 03:22:29.252629'),
 'top: 3 year: 2016': Timedelta('9 days 17:26:12.836868'),
 'top: 3 year: 2017': Timedelta('13 days 13:04:12.375066'),
 'top: 3 year: 2018': Timedelta('16 days 02:52:32.636896'),
 'top: 3 year: 2019': Timedelta('16 days 16:48:17.268487'),
 'top: 5 year: 2014': Timedelta('15 days 12:48:20.794223'),
 'top: 5 year: 2015': Timedelta('13 days 16:41:29.913544'),
 'top: 5 year: 2016': Timedelta('10 days 09:30:28.820960'),
 'top: 5 year: 2017': Timedelta('16 days 07:55:07.396852'),
 'top: 5 year: 2018': Timedelta('16 days 07:04:08.598386'),
 'top: 5 year: 2019': Timedelta('16 days 11:49:09.117520'),
 'top: 10 year: 2014': Timedelta('13 days 08:44:18.663028'),
 'top: 10 year: 2015': Timedelta('12 days 02:17:04.760728'),
 'top: 10 year: 2016': Timedelta('10 days 09:36:04.492801'),
 'top: 10 year: 2017': Timedelta('15 days 17:00:28.346456'),
 'top: 10 year: 2018': Timedelta('15 

In [25]:
[('FY{}:<br>{:,} requests'.format(year,val)) for val,year in zip([value for value in df.groupby('fiscal_year_requested')['wo_id'].count()],
                   [value for value in df.groupby('fiscal_year_requested')['wo_id'].count().index])]

['FY2014:<br>12,209 requests',
 'FY2015:<br>14,522 requests',
 'FY2016:<br>16,679 requests',
 'FY2017:<br>17,873 requests',
 'FY2018:<br>16,593 requests',
 'FY2019:<br>6,186 requests']

In [26]:
for val,year in zip([value for value in df.groupby('fiscal_year_requested')['wo_id'].count()],
                   [value for value in df.groupby('fiscal_year_requested')['wo_id'].count().index]):
    print('{}: {}'.format(year, val))

2014: 12209
2015: 14522
2016: 16679
2017: 17873
2018: 16593
2019: 6186


In [27]:

problems = filtered_df['prob_type'].value_counts().index.tolist()
prob_type_counts, prob_type_avg_duration = [],[]
for prob in problems:
    prob_type_counts.append(filtered_df[filtered_df['prob_type'] == prob]['wo_id'].count())
    prob_type_avg_duration.append(filtered_df[filtered_df['prob_type'] == prob]['duration'].mean().days)
    
    

# create list for sizing bubbles on chart based on pct of the work orders 
# in that problem type that exceed the average duration for that type
pct_workorders_exceeding_mean_duration_for_type = []
for prob in problems:
    avg = filtered_df[filtered_df['prob_type'] == prob]['duration'].mean().days
    number_exceding_mean_duration = filtered_df[(filtered_df['prob_type'] == prob) & 
                     (filtered_df['duration'].dt.days > avg)]['duration'].count()
    count_ = filtered_df[(filtered_df['prob_type'] == prob)]['wo_id'].count()
    pct_workorders_exceeding_mean_duration_for_type.append(number_exceding_mean_duration / count_ * 100)

    
x = prob_type_counts
y = prob_type_avg_duration

data = [
    go.Histogram2dContour(
        x = prob_type_counts,
        y = prob_type_avg_duration,
        name = '',
        colorscale = 'Blues',
        reversescale = True,
        xaxis = 'x',
        yaxis = 'y'
    ),
    
    go.Scatter(
        x = prob_type_counts,
        y = prob_type_avg_duration,
        xaxis = 'x',
        yaxis = 'y',
        name = 'problem type',
        mode = 'markers',
        hoverinfo = 'text',
        text = ['{}'.format(prob) for prob in problems],
        marker = dict(
            color = 'red',
            size = [val / 4 for val in pct_workorders_exceeding_mean_duration_for_type]),
        opacity = .5
    ),
    go.Histogram(
        y = prob_type_avg_duration,
        xaxis = 'x2',
        nbinsy = 25,
        name = 'avg days',
        marker = dict(
            color = '#535556')
                ),
    go.Histogram(
        x = prob_type_counts,
        yaxis = 'y2',
        name = 'type volume',
        nbinsx = 25,
        marker = dict(
            color = '#535556')
                )
]

layout = go.Layout(
    title = 'Duration & Request Volume<br>Distribution Density by Type',
    autosize = False,
    xaxis = dict(
        title = 'request volume<br>(by problem type)',
        zeroline = False,
        domain = [0,0.85],
        showgrid = False
    ),
    yaxis = dict(
        title = 'avg duration (days)',
        zeroline = False,
        domain = [0,0.85],
        showgrid = False
    ),
    xaxis2 = dict(
        zeroline = False,
        domain = [0.85,1],
        showgrid = False
    ),
    yaxis2 = dict(
        zeroline = False,
        domain = [0.85,1],
        showgrid = False
    ),
    height = 600,
    width = 600,
    bargap = 0,
    hovermode = 'closest',
    showlegend = False, 
    annotations = [dict(
        x = 1,
        y = 1,
        xref = 'paper',
        yref = 'paper',
        font = dict(color = 'red'),
        text = 'bubble size = % of<br>durations > mean',
        showarrow = False)]
)

fig = go.Figure(data=data,layout=layout)
pyo.iplot(fig)

In [28]:
# work order duration distribution by technician who completes job

In [29]:
pct_workorders_exceeding_mean_duration_for_type = {}

for prob in problems:
    avg = filtered_df[filtered_df['prob_type'] == prob]['duration'].mean().days
    number_exceding_mean_duration = filtered_df[(filtered_df['prob_type'] == prob) & 
                     (filtered_df['duration'].dt.days > avg)]['duration'].count()
    count_ = filtered_df[(filtered_df['prob_type'] == prob)]['wo_id'].count()
    #pct_workorders_exceeding_mean_duration_for_type.append(number_exceding_mean_duration / count_ * 100)
    pct_workorders_exceeding_mean_duration_for_type[prob] = number_exceding_mean_duration / count_ * 100